In [85]:
import numpy as np
import pandas as pd
from tensorflow import keras
from sklearn.preprocessing import MinMaxScaler

In [83]:
train_data = pd.read_csv("./titanic/train.csv", index_col=False, header=0)
train_data = train_data.drop(columns=['Name', 'PassengerId', 'Ticket', 'Cabin'])

# 填充上船地点，nan用平均ASCII码值
embarked_count = train_data['Embarked'].value_counts()
emb_fill = sum(map(lambda ch: embarked_count[ch] * ord(ch) / sum(embarked_count),
                   embarked_count.index))
embarked_dic = {key: ord(key) for key in embarked_count.index}
train_data['Embarked'] = train_data['Embarked'].apply(lambda ch: embarked_dic.get(ch, emb_fill))

# 确认其他数据无nan
print(train_data['Pclass'].isnull().sum(),
      train_data['Sex'].isnull().sum(),
      train_data['SibSp'].isnull().sum(),
      train_data['Parch'].isnull().sum(),
      train_data['Fare'].isnull().sum())

# 填充年龄
female_ave = train_data[train_data['Sex'] == 'female']['Age'].mean()
male_ave = train_data[train_data['Sex'] == 'male']['Age'].mean()
def fill_sex(sex_, value_):
    if pd.isnull(value_):
        if sex_ == 'male':
            return male_ave
        return female_ave
    else:
        return value_
train_data = train_data.values
for i in range(len(train_data)):
    train_data[i][3] = fill_sex(train_data[i][2], train_data[i][3])
train_data[:, 2] = np.array(list(map(lambda ch: 1 if ch == 'male' else -1 , train_data[:, 2])))

0 0 0 0 0


In [84]:
train_x = train_data[:, 1:]
train_label = train_data[:, 0]
scaler = MinMaxScaler(feature_range=(0, 1))
train_x = scaler.fit_transform(train_x)

In [ ]:
network = keras.